In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')
from functools import reduce

In [ ]:
MIN = -1e-9
def counterClockwise(p1, p2, P):
    x = (p2[0]-p1[0])*(P[1]-p2[1])
    y = (p2[1]-p1[1])*(P[0]-p2[0])
    if x-y >= MIN: return True
    else: return False

In [ ]:
def Clockwise(P):
    n = len(P)
    ans = 0
    for i in range(n):
        ans += (P[(i + 1) % n][0] - P[i][0]) * (P[(i + 1) % n][1] + P[i][1])
    if ans > 0: return True
    else: return False

In [ ]:
def inside(p1, p2, p3, p4):
    if p1==p4 or p2==p4 or p3==p4 : return False
    if not counterClockwise(p3, p1, p4): return False
    elif not counterClockwise(p1, p2, p4): return False
    elif not counterClockwise(p2, p3, p4): return False
    return True

In [ ]:
def ear(pontos, adj, x):
    t = [pontos[adj[x][0]], pontos[x], pontos[adj[x][1]]]
    for p in pontos:
        if inside(t[0], t[1], t[2], p): return False
    if not counterClockwise(t[0],t[1],t[2]): return False
    return True

In [ ]:
def earClipping(pontos):
    
    if Clockwise(pontos):
        pontos.reverse()
    process = pontos.copy()
    n = len(pontos)
    adj = [[n-1, 1]]
    for i in range(n-2): adj.append([i, i+2])
    adj.append([n-2, 0])
    
    triang = []
    polig = []
    
    ears = set()
    for i in range(len(pontos)):
        if ear(pontos, adj ,i):
            ears.add(i)
    
    while len(process) > 2:
        p = next(iter(ears), None)
        ears.remove(p)
        process.remove(pontos[p])

        pX = adj[p][0] 
        pY = adj[p][1]
        adj[pY][0] = pX
        adj[pX][1] = pY
        
        k = [pX, p, pY]
        l = [pontos[pX], pontos[p], pontos[pY]]
        
        triang.append(k)
        polig.append(l)

        if ear(pontos, adj, pX):
            ears.add(pX)
        else:
            for x in ears:
                if x == pX:
                    ears.remove(pX)
                    break

        if ear(pontos, adj, pY):
            ears.add(pY)
        else:
            for x in ears:
                if x == pY:
                    ears.remove(pY)
                    break
            
        hv.output(hv.Polygons([pontos[pX], pontos[p], pontos[pY]]).opts(color = "white") * hv.Polygons([process]).opts(color = "black") * hv.Points([pontos[p]]).opts(color = 'green', size = 15))
    hv.output(hv.Polygons(polig))
    
    return triang
    

In [ ]:
def output(pontos, triang):
    Xcolors = {0: "black", 1: "red", 2: "green", 3: "blue"}
    plot_point = [hv.Points(i["point"]).opts(size = 15,color = Xcolors[i["color"]]) for i in pontos]
    ans = plot_point[0]
    l = len(plot_point)
    for i in range(1, l): ans *= plot_point[i]
    hv.output(triang * ans)

In [ ]:
import queue
def color(pontos, triangulo):
    n = len(pontos)
    triangulos = hv.Polygons([pontos[t[0]], pontos[t[1]], pontos[t[2]]] for t in triangulo)
    pontos = [{"point": pontos[i], "color": 0} for i in range(n)]
    
    q = queue.Queue()
    q.put(0)
    vis = [0 for i in range(n)]
    colors = [0 for i in range(n)]
    vis[0] = 1
    for i in range(3):
        pontos[triangulo[0][i]]["color"] = i+1
        colors[triangulo[0][i]] = i+1
    
    output(pontos, triangulos)
    
    while not q.empty():
        this = q.get()
        for j in range(n-2):
            if not vis[j]:
                sa = [k for k in triangulo[this] if k in triangulo[j]]
                no = [k for k in triangulo[this]+triangulo[j] if k not in sa]
                if len(sa) == 2:
                    vis[j] = 1
                    colors[no[1]] = colors[no[0]]
                    pontos[no[1]]["color"] = colors[no[0]]
                    q.put(j)
        
        output(pontos, triangulos)
    output(pontos, triangulos)
    
    return None

In [ ]:
crista = [[(0.5*a,10), (0.5*(a+1),2), (0.5*(a+2),2)] for a in range(1,25,3)]
ponto = [(0,0)] + list(reduce(lambda x,y: x+y, crista)) + [(12,0),(0,0)]
del ponto[-3]
del ponto[-3]
ponto2 = [(0,0), (1,3), (4,-2),(6.5,4),(2,4.5),(7.5,7),(1.5,7.5),(0.6,5),(-0.8,6),(0,0)]
ponto3 = [(np.cos((2*k*np.pi)/8),np.sin((2*k*np.pi)/8)) for k in range(1,9)]
ponto4 = [(3,4), (2,2), (3.5025,1.02125), (3.8025,2.64125), (4.7825,1.22125), (6.2225,1.20125), (6.5225,2.42125), (5.6025,3.58125), (5.0625,2.52125), (4.3425,3.48125), (5.3825,4.68125)]

In [ ]:
color(ponto, earClipping(ponto))

In [ ]:
color(ponto2, earClipping(ponto2))

In [ ]:
color(ponto3, earClipping(ponto3))

In [ ]:
color(ponto4, earClipping(ponto4))